# **MCP Client**

MCP is a standardised way to integrate tools with your applications. As defined by Anthropic:
> **MCP is an open protocol that standardizes how your LLM Applicaitons connect to and work with your tools and data sources.**

Problem with the Tools:
- N*M Client-side Integrations
- More Maintanance
- Time Consuming

With MCP:
- Server do the heavy lifting
- Client simply plugin and play

### **Connecting your MCP Client with Other MCP Servers**

Check out:
- https://github.com/punkpeye/awesome-mcp-servers
- https://glama.ai/mcp/servers
- https://mcp.so/servers

In [8]:
from langchain_openai import ChatOpenAI

# Setup API Key
f = open('keys/.openai_api_key.txt')
OPENAI_API_KEY = f.read()

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                 model="gpt-4o-mini",
                 temperature=0.0)

In [9]:
# !pip install langchain-mcp-adapters

In [10]:
from langchain_mcp_adapters.client import MultiServerMCPClient

Each server must include 'transport' with one of: 'stdio', 'sse', 'websocket', 'http'.

In [24]:
# ! pip install uv

In [25]:
client = MultiServerMCPClient(
    {
        "time": {
          "transport": "stdio",
          "command": "uvx",
          "args": [
            "mcp-server-time",
            "--local-timezone=America/New_York"
          ]
        }
    }
)

In [26]:
tools = await client.get_tools()

tools

[StructuredTool(name='get_current_time', description='Get current time in a specific timezones', args_schema={'type': 'object', 'properties': {'timezone': {'type': 'string', 'description': "IANA timezone name (e.g., 'America/New_York', 'Europe/London'). Use 'America/New_York' as local timezone if no timezone provided by the user."}}, 'required': ['timezone']}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x116cfc900>),
 StructuredTool(name='convert_time', description='Convert time between timezones', args_schema={'type': 'object', 'properties': {'source_timezone': {'type': 'string', 'description': "Source IANA timezone name (e.g., 'America/New_York', 'Europe/London'). Use 'America/New_York' as local timezone if no source timezone provided by the user."}, 'time': {'type': 'string', 'description': 'Time to convert in 24-hour format (HH:MM)'}, 'target_timezone': {'type': 'string', 'description': "Target IANA timezone 

In [27]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=tools
)

In [29]:
response = await agent.ainvoke({"messages": "What time is it?"})

for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What time is it?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_oMlVxNqgEkD2qctfETSyk0Mr)
 Call ID: call_oMlVxNqgEkD2qctfETSyk0Mr
  Args:
    timezone: America/New_York
================================= Tool Message =================================
Name: get_current_time

[{'type': 'text', 'text': '{\n  "timezone": "America/New_York",\n  "datetime": "2025-12-18T14:06:39-05:00",\n  "day_of_week": "Thursday",\n  "is_dst": false\n}', 'id': 'lc_68cc2e6d-cca1-4f7c-8997-6cfad38abd41'}]
================================== Ai Message ==================================

The current time in New York is 2:06 PM on Thursday, December 18, 2025.


In [30]:
response = await agent.ainvoke({"messages": "What time is it in India?"})

for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

What time is it in India?
================================== Ai Message ==================================
Tool Calls:
  get_current_time (call_f0wa1xhAamDeH522gEH2z82p)
 Call ID: call_f0wa1xhAamDeH522gEH2z82p
  Args:
    timezone: Asia/Kolkata
================================= Tool Message =================================
Name: get_current_time

[{'type': 'text', 'text': '{\n  "timezone": "Asia/Kolkata",\n  "datetime": "2025-12-19T00:39:23+05:30",\n  "day_of_week": "Friday",\n  "is_dst": false\n}', 'id': 'lc_c5b932ee-ceaf-48a1-9349-25bdef564a90'}]
================================== Ai Message ==================================

The current time in India (Asia/Kolkata) is 12:39 AM on December 19, 2025.


## **Building Flight Agent**

In [1]:
from langchain_mcp_adapters.client import MultiServerMCPClient

Each server must include 'transport' with one of: 'stdio', 'sse', 'websocket', 'http'.

In [2]:
client = MultiServerMCPClient(
    {
        "my_travel_server": {
          "transport": "streamable_http",
          "url": "https://mcp.kiwi.com"
        }
    }
)

tools = await client.get_tools()

tools

[StructuredTool(name='search-flight', description='\n# Search for a flight\n\n## Description\n\nUses the Kiwi API to search for available flights between two locations on a specific date.\n\n## How it works\n\nThe tool will:\n1. Search for matching locations to resolve airport codes\n2. Find available flights for the specified route and date range\n\n## Method\n\nCall this tool whenever a user wants to search for flights, regardless of whether they provided exact airport codes or just city names.\n\nYou should display the returned results in a markdown table format: Group the results by price (those who are the cheapest), duration (those who are the shortest, i.e. have the smallest \'totalDurationInSeconds\') and the rest (those that could still be interesting).\n\nAlways display for each flight in order:\n  - In the 1st column: The departure and arrival airports, including layovers (e.g. "Paris CDG → Barcelona BCN → Lisbon LIS")\n  - In the 2nd column: The departure and arrival dates 

In [4]:
from langchain_openai import ChatOpenAI

# Setup API Key
f = open('keys/.openai_api_key.txt')
OPENAI_API_KEY = f.read()

llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY,
                 model="gpt-4o-mini",
                 temperature=0.0)

In [6]:
from langchain.agents import create_agent

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt="You are a travel agent. Dont ask any follow-up questions."
)

In [14]:
response = await agent.ainvoke({"messages": "Get me a direct flight from NYC to India on January 3rd 2026."})

for msg in response["messages"]:
    msg.pretty_print()

================================ Human Message =================================

Get me a direct flight from NYC to India on January 3rd 2026.
================================== Ai Message ==================================
Tool Calls:
  search-flight (call_fqTrt70GVqexG9oMqIecz7Et)
 Call ID: call_fqTrt70GVqexG9oMqIecz7Et
  Args:
    flyFrom: NYC
    flyTo: India
    departureDate: 03/01/2026
    passengers: {'adults': 1}
    sort: price
================================= Tool Message =================================
Name: search-flight

[{'type': 'text', 'text': '[\n  {\n    "flyFrom": "EWR",\n    "flyTo": "IND",\n    "cityFrom": "New York",\n    "cityTo": "Indianapolis",\n    "departure": {\n      "utc": "2026-01-03T10:00:00.000Z",\n      "local": "2026-01-03T05:00:00.000"\n    },\n    "arrival": {\n      "utc": "2026-01-03T12:13:00.000Z",\n      "local": "2026-01-03T07:13:00.000"\n    },\n    "totalDurationInSeconds": 7980,\n    "durationInSeconds": 7980,\n    "price": 70,\n    "de